In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [ ]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
sys.path.append('../utils')
import Reentry
from scipy.integrate import solve_ivp
from Scvx_tf_free import Scvx_tf_free
from Scaling import TrajectoryScaling

In [ ]:
ix = 6
iu = 2
tf = 3.4
N = 99
delT = tf/N
max_iter = 10

In [ ]:
Re = 6371*1000
ge = 9.806

scl_d = Re;
scl_v = np.sqrt(ge*Re);

x0  = 0.0/scl_d; 
y0  = 0.0/scl_d; 
h0  = 122e3/scl_d;
v0  = 7315.2/scl_v; 
gam0 = np.deg2rad(-1.5); 
thet0 = np.deg2rad(4); 

xi = np.array([x0,y0,h0,v0,gam0,thet0])
xf = np.array([100,100,100,10,0,0])
print(xi,xf)

In [ ]:
x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf

u0 = np.zeros((N+1,iu))
u0[:,0] = 1.7 # setting not as zero is pretty critical
u0[:,1] = 0.0

In [ ]:
myModel = Reentry.Reentry('Hello',ix,iu,'numeric_central')

# test continuous

In [ ]:
x = np.expand_dims(np.array([1,2,3,4,5,6]),0)
u = np.expand_dims(np.array([100,20]),0)
print_np(x)

In [ ]:
A,B = myModel.diff_numeric(x,u,discrete=False)

In [ ]:
A

# test discrete

In [ ]:
import scipy.io
path = '/Users/keemtw/Documents/Code/Personal/reentryopt-master-purna/scp_tools/testing_foh/zbar_save.mat'
mat = scipy.io.loadmat(path)

In [ ]:
zbar = np.array(mat['zbar_']).T

In [ ]:
zbar,u0

In [ ]:
print_np(zbar)
print_np(u0)

In [ ]:
start = time.time()
A,Bm,Bp,s,z,x_prop = myModel.diff_discrete_foh(zbar[:N],u0,delT,tf)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
A_,Bm_,Bp_,s_,z_,x_prop_ = myModel.diff_discrete_foh_serial(zbar[:N],u0,delT,tf)
end = time.time()
print(end - start)

In [ ]:
for mat,mat_ in zip([A,Bm,Bp,s,z,x_prop],[A_,Bm_,Bp_,s_,z_,x_prop_]) :
    print(np.max(np.abs(mat - mat_)))

In [ ]:
print_np(A_)

In [ ]:
print_np(Bm_)